In [1]:
"""
Autohr: Sergi Mas-Pujol
Last update: 12/01/2021

Python version: 3.6
"""

'\nAutohr: Sergi Mas-Pujol\nLast update: 12/01/2021\n\nPython version: 3.6\n'

In [2]:
import pandas as pd

from utils_timeProcessing import from_YYYYMMDD_to_DDMMYYYY_given_separator

In [3]:
list_files_nreg_to_convert_to_csv = ['RegPlan_1906.nreg', 'RegPlan_1907.nreg', 'RegPlan_1908.nreg', 'RegPlan_1909.nreg']

convertion_regulations = {'C': 'ATC CAPACITY',
                          'G': 'AERODROME CAPACITY',
                          'W': 'WEATHER',
                          'T': 'EQUIPMENT (ATC)',
                          'E': 'EQUIPMENT (NOT ATC)',
                          'I': 'INDUSTRIAL ACTION',
                          'M': 'MILITARY ACTIVITY',
                          'R': 'ATC ROUTING',
                          'S': 'ATC STAFFING',
                          'V': 'ENVIRONMENTAL ISSUES',
                          'D': 'DE-ICING',
                          'P': 'SPECIAL EVENT', 
                          'O': 'OTHER REASON',
                          'U': 'UNKNOWN REASON'}

regulations_csv = pd.DataFrame(columns=['date_DDMMYYYY', 
                                        'regulation_ID', 
                                        'traffic_volume',
                                        'start_regulations_HHMMSS', 'end_regulations_HHMMSS', 
                                        'regulation_reason'], )

In [4]:
for reg_file in list_files_nreg_to_convert_to_csv:
    reg_plan = open('./Exports_RNEST/'+reg_file, 'r')
    print(reg_file)

    while True:
        # Initialize the dictionary to store information fot the next line in the dataFrama
        next_sample_data_frame = {}

        # Iterate the main lines (regulation)
        next_line = reg_plan.readline()
        
        if next_line == '':  # If next line is emty -> End of the file --> Use the next file: RegPlan_XXXX.nreg
            break
        
        date_YYYYMMDD = next_line.split(' ')[0]
        date_DDMMYYYY = date_YYYYMMDD[6:8] + '/' + date_YYYYMMDD[4:6] + '/' + date_YYYYMMDD[0:4]  # Changing format
        regulation_ID = next_line.split(' ')[1]
        traffic_volume = next_line.split(' ')[2]
        regulation_reason = next_line.split(' ')[-2]+'-'+convertion_regulations[next_line.split(' ')[-2]]

        next_sample_data_frame['date_DDMMYYYY'] = date_DDMMYYYY
        next_sample_data_frame['regulation_ID'] = regulation_ID
        next_sample_data_frame['traffic_volume'] = traffic_volume
        next_sample_data_frame['regulation_reason'] = regulation_reason

        num_intervals = int(next_line.split(' ')[-1])

        # Iterate the sub-lines (timestamps regulation)
        regulated_intervals_start = []
        regulated_intervals_end = []
        for _ in range(num_intervals):
            next_line = reg_plan.readline()
            regulated_intervals_start.append(next_line.split(' ')[0])
            regulated_intervals_end.append(next_line.split(' ')[1])

        # Changing format start/end timestamp regulation
        start_regulations_HHMMSS = regulated_intervals_start[0].split(':')[0] + regulated_intervals_start[0].split(':')[1] + '00'
        regulated_intervals_end = regulated_intervals_end[-1].split(':')[0] + regulated_intervals_end[-1].split(':')[1] + '00'

        next_sample_data_frame['start_regulations_HHMMSS'] = start_regulations_HHMMSS
        next_sample_data_frame['end_regulations_HHMMSS'] = regulated_intervals_end

        regulations_csv = regulations_csv.append(next_sample_data_frame, ignore_index=True)

RegPlan_1906.nreg
RegPlan_1907.nreg
RegPlan_1908.nreg
RegPlan_1909.nreg


In [5]:
regulations_csv

,date_DDMMYYYY,regulation_ID,traffic_volume,start_regulations_HHMMSS,end_regulations_HHMMSS,regulation_reason
0,25/05/2019,YD6WH25E,MASD6WH,053000,063000,C-ATC CAPACITY
1,26/05/2019,FL4BEH26,FL4BEH,080000,084000,C-ATC CAPACITY
2,26/05/2019,YBOLN26,MASBOLN,152000,153000,C-ATC CAPACITY
3,27/05/2019,FL2DWM27,FL2DWM,123000,140000,C-ATC CAPACITY
4,27/05/2019,FL4DWH27,FL4DWH,060000,080000,C-ATC CAPACITY
...,...,...,...,...,...,...
588,08/09/2019,YBALB08,MASBALB,140000,170000,C-ATC CAPACITY
589,08/09/2019,YBOLN08,MASBOLN,140000,141000,C-ATC CAPACITY
590,08/09/2019,YBOLN08M,MASBOLN,112000,170000,C-ATC CAPACITY
591,10/09/2019,YHAMRU10,MASHAMRU,190000,210000,M-MILITARY ACTIVITY


In [6]:
regulations_csv.to_csv('./20190523_20190911_REGULATIONS_MUAC.csv', sep=';', index=False)